# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 0.9121232585461544),
 (1, 0.9121232585461544),
 (2, 0.9121232585461544),
 (3, 0.9121232585461544),
 (4, 0.9121232585461544)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('is', ['0', '1', '2']),
 ('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('banana', 2)]),
 (1, [('a', 2), ('is', 18), ('it', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.002443364077132637),
   (None, 0.02286481564053089),
   (None, 0.04105261629197765),
   (None, 0.05928769663753364),
   (None, 0.0669979938812334),
   (None, 0.07806151703058428),
   (None, 0.137021086943383),
   (None, 0.13754484522034094),
   (None, 0.2193752216769136),
   (None, 0.31021373046467526),
   (None, 0.35326770916740624),
   (None, 0.3825256180133989),
   (None, 0.38917568616530496),
   (None, 0.4090424323849997)]),
 (1,
  [(None, 0.5077648517223402),
   (None, 0.5375114052052647),
   (None, 0.5443796796598657),
   (None, 0.5614722810777465),
   (None, 0.5748039066550942),
   (None, 0.5794753685576345),
   (None, 0.6125317404348489),
   (None, 0.6229956473240211),
   (None, 0.6691955041177795),
   (None, 0.7077406139736274),
   (None, 0.8685205141336637),
   (None, 0.8789897412075368),
   (None, 0.9685645976805717),
   (None, 0.9738367206974853),
   (None, 0.9964761383958918),
   (None, 0.9994908016416101)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
class Book(NamedTuple):
  id: int
  year: int
  title: str
  genre: str

In [22]:
input_collection_praxis = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel"),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy")
]

In [32]:
def MAP_FIND_MAX(_, row:NamedTuple):
    yield ("", row)

# поиск максимального года публикации книги (самой новой)
def REDUCE_FIND_MAX(_, rows: Iterator[NamedTuple]):
    max_year = 0
    max_year_book = None
    for row in rows:
        if row.year > max_year:
            max_year = row.year
            max_year_book = row
    yield max_year_book


def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis]


output = MapReduce(RECORDREADER, MAP_FIND_MAX, REDUCE_FIND_MAX)
output = list(output)
output

[Book(id=2, year=1995, title='His Dark Materials', genre='fantasy')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [33]:
def MAP_FIND_AVG(_, row:NamedTuple):
    yield ("", row)

# поиск среднего года публикации
def REDUCE_FIND_AVG(_, rows: Iterator[NamedTuple]):
    sum = 0
    for row in rows:
        sum += row.year
    yield (sum / len(rows))

def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis]


output = MapReduce(RECORDREADER, MAP_FIND_AVG, REDUCE_FIND_AVG)
output = list(output)
output

[1935.25]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [34]:
def groupbykey_sorted(iterable):
  # Сортируем элементы по ключу
    iterable = sorted(iterable, key=lambda elem: elem[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def MapReduce_with_sorted(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey_sorted(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

In [35]:
output = MapReduce_with_sorted(RECORDREADER, MAP_FIND_MAX, REDUCE_FIND_MAX)
output = list(output)
print("Максимальное значение ряда: " + str(output))

output = MapReduce_with_sorted(RECORDREADER, MAP_FIND_AVG, REDUCE_FIND_AVG)
output = list(output)
print("Арифметическое среднее:" + str(output))

Максимальное значение ряда: [Book(id=2, year=1995, title='His Dark Materials', genre='fantasy')]
Арифметическое среднее:[1935.25]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [27]:
input_collection_praxis_duplicates = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel"),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy"),
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy")
]

In [315]:
maps = 3

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (id, book) in enumerate(split):
            yield (id, book)

    split_size = int(np.ceil(len(input_collection_praxis_duplicates) / maps))
    for i in range(0, len(input_collection_praxis_duplicates), split_size):
        yield RECORDREADER(input_collection_praxis_duplicates[i : i + split_size])

# Группировка по id
def MAP_UNIQUE(_, row:NamedTuple):
    yield (row.id, row)

# Возвращение одной строки из множества с одинаковыми id
def REDUCE_UNIQUE(id: int, rows: Iterator[NamedTuple]):
    yield rows[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

8 key-value pairs were sent over a network.
[(0, [Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy'), Book(id=1, year=1813, title='Pride and Prejudice', genre='novel'), Book(id=2, year=1995, title='His Dark Materials', genre='fantasy'), Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre='comedy')])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [45]:
input_collection_praxis = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel"),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy")
]

In [55]:
def MAP_SELECTION(_, row:NamedTuple):
  if row.genre == "fantasy":
      yield (row, row)

def REDUCE_SELECTION(row:str, rows:Iterator[NamedTuple]):
    yield row

def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis]

output = MapReduce(RECORDREADER, MAP_SELECTION, REDUCE_SELECTION)
output = list(output)
output

[Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy'),
 Book(id=2, year=1995, title='His Dark Materials', genre='fantasy')]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [74]:
input_collection_praxis = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel"),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel")
]

In [75]:
S = {"fantasy"}

def MAP_PROJECTION(_, row:NamedTuple):
  # фильтрация на основе множества S
    if row.genre not in S:
        yield (row, row)

def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
  yield row

def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[Book(id=1, year=1813, title='Pride and Prejudice', genre='novel'),
 Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre='comedy')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [76]:
input_collection_praxis_1 = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel")
]

In [77]:
input_collection_praxis_2 = [
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel")
]

In [93]:
input_collection_praxis_union = input_collection_praxis_1 + input_collection_praxis_2

def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])

def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis_union]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy'),
  Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy')),
 (Book(id=1, year=1813, title='Pride and Prejudice', genre='novel'),
  Book(id=1, year=1813, title='Pride and Prejudice', genre='novel')),
 (Book(id=2, year=1995, title='His Dark Materials', genre='fantasy'),
  Book(id=2, year=1995, title='His Dark Materials', genre='fantasy')),
 (Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre='comedy'),
  Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre='comedy'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [104]:
def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE_INTERSECTION(_, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

def RECORDREADER():
    return [(b.id, b) for b in input_collection_praxis_union]

output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[Book(id=1, year=1813, title='Pride and Prejudice', genre='novel'),
  Book(id=1, year=1813, title='Pride and Prejudice', genre='novel')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [121]:
input_collection_praxis_1 = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel")
]

In [148]:
input_collection_praxis_2 = [
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy"),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy"),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel")
]

In [150]:
def MAP_DIFFERENCE(relation: int, row: NamedTuple):
    yield (row, relation)

def REDUCE_DIFFERENCE(rows: Iterator[NamedTuple], relations: list[int]):
    if relations == [0]:
      yield (rows, rows)

def RECORDREADER():
    return [(0, a) for a in input_collection_praxis_1] + [(1, b) for b in input_collection_praxis_2]

output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[(Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy'),
  Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [151]:
class Book(NamedTuple):
  id: int
  year: int
  title: str
  genre: str
  autor_id: int

In [157]:
class Autor(NamedTuple):
  id: int
  name: str

In [154]:
input_collection_books = [
    Book(id=0, year=1954, title="The Lord of the Rings", genre="fantasy", autor_id=0),
    Book(id=1, year=1813, title="Pride and Prejudice", genre="novel", autor_id=1),
    Book(id=2, year=1995, title="His Dark Materials", genre="fantasy", autor_id=2),
    Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre="comedy", autor_id=3),
    Book(id=4, year=1811, title="Sense and Sensibility", genre="novel", autor_id=1)
]

In [158]:
input_collection_autors = [
    Autor(id=0, name="John Ronald Reuel Tolkien"),
    Autor(id=1, name="Jane Austen"),
    Autor(id=2, name="Philip Pullman"),
    Autor(id=3, name="Douglas Adams")
]

In [164]:
def MAP_JOIN(id: int, row: NamedTuple):
    yield (id, row)

def REDUCE_JOIN(id: int, rows: Iterator[NamedTuple]):
    books = []
    for row in rows:
        if isinstance(row, Book):
            books.append(row)
        else:
            autor = row
    for row in rows:
        if isinstance(row, Book):
            yield (row, row.autor_id, autor)

def RECORDREADER():
    return [(book.autor_id, book) for book in input_collection_books] + [(autor.id, autor) for autor in input_collection_autors]

output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
join_input = output
join_input

[(Book(id=0, year=1954, title='The Lord of the Rings', genre='fantasy', autor_id=0),
  0,
  Autor(id=0, name='John Ronald Reuel Tolkien')),
 (Book(id=1, year=1813, title='Pride and Prejudice', genre='novel', autor_id=1),
  1,
  Autor(id=1, name='Jane Austen')),
 (Book(id=4, year=1811, title='Sense and Sensibility', genre='novel', autor_id=1),
  1,
  Autor(id=1, name='Jane Austen')),
 (Book(id=2, year=1995, title='His Dark Materials', genre='fantasy', autor_id=2),
  2,
  Autor(id=2, name='Philip Pullman')),
 (Book(id=3, year=1979, title="The Hitchhiker's Guide to the Galaxy", genre='comedy', autor_id=3),
  3,
  Autor(id=3, name='Douglas Adams'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [166]:
def MAP_GROUP(id: int, row_b: NamedTuple, row_c: NamedTuple):
    yield (id, row_b)

def REDUCE_GROUP(id: int, rows: Iterator[NamedTuple]):
    yield (id, len(rows))

def RECORDREADER():
    return [(autor_id, book, autor) for book, autor_id, autor in join_input]

output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

[(0, 1), (1, 2), (2, 1), (3, 1)]

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [322]:
mat = np.ones((5, 4))
vec = np.random.rand(4)

def MAP(coordinates: (int, int), value: int, chunk_start: int, chunk_end: int):
    i, j = coordinates
    vec_chunk = vec[chunk_start:chunk_end]
    if j >= chunk_start and j < chunk_end:
        yield (i, value * vec_chunk[j - chunk_start])

def REDUCE(i: int, products: Iterator[NamedTuple]):
    yield (i, sum(products))

def RECORDREADER():
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            yield ((i, j), mat[i, j])

def MapReduceWithChunks(recordreader, map_func, reduce_func, chunk_size=2):
    results = {}
    # Обрабатываем вектор по частям
    for chunk_start in range(0, len(vec), chunk_size):
        chunk_end = min(chunk_start + chunk_size, len(vec))

        for coordinates, value in recordreader():
            for result in map_func(coordinates, value, chunk_start, chunk_end):
                i, product = result
                if i not in results:
                    results[i] = []
                results[i].append(product)

    for i, products in results.items():
        yield (i, sum(products))


output = MapReduceWithChunks(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.5631593847998777),
 (1, 1.5631593847998777),
 (2, 1.5631593847998777),
 (3, 1.5631593847998777),
 (4, 1.5631593847998777)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [323]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [339]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
      yield ((i, k), small_mat[i, j] * w)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

Проверьте своё решение

In [372]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [341]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [360]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range(I):
    for j in range(J):
      for k in range(K):
        yield (((i,j), small_mat[i,j]), ((j,k), big_mat[j,k]))

def MAP(e1, e2):
  (i, j), v1 = e1
  (j, k), v2 = e2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

In [373]:
# CHECK THE SOLUTION
solution = MapReduce(RECORDREADER, MAP, REDUCE)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [401]:
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)

maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(part):
      for i in part:
        for j in range(J):
          for k in range(K):
            yield (((i,j), small_mat[i,j]), ((j,k), big_mat[j,k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


def MAP(e1, e2):
  (i, j), v1 = e1
  (j, k), v2 = e2
  yield ((i, k), v1 * v2)


def REDUCE(key, values):
  (i, k) = key
  yield (key, sum(values))

In [402]:
reference_solution = np.matmul(small_mat, big_mat)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

full_solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        full_solution += [element]

np.allclose(reference_solution, asmatrix(full_solution)) # should return true

240 key-value pairs were sent over a network.


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Решение работает, потому что каждая пара элементов, генерируемая RECORDREADER, уникально определяется по своим индексам.